In [1]:
import cv2
import datetime
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, AveragePooling2D, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard

# augmentation
train_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Train'
test_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Test'
val_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation'

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=32
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_generator = train_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=32
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = train_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=32
)

mobilenet = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(128, 128, 3)
)

for layer in mobilenet.layers:
    layer.trainable = False

model = Sequential()
model.add(mobilenet)
model.add(Flatten())
model.add(Dense(2, activation='sigmoid'))


model.compile(optimizer="adam", loss="binary_crossentropy",
              metrics=["acc"])


log_dir = "logs/fit/test-sigmoid" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tb = TensorBoard(histogram_freq=1, write_graph=True, write_grads=True, log_dir=log_dir)


history = model.fit(
    x=train_generator,
    steps_per_epoch=len(train_generator) // 32,
    epochs=20,
    validation_data=val_generator,
    # validation_steps=len(val_generator) // 32,
    # validation_split=0.01
    callbacks=[tb]
)

# model.evaluate(test_generator)

model.save('masknet-sigmoid.h5')




Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/20
1/9 [==>...........................] - ETA: 0s - loss: 1.0849 - acc: 0.5625WARNING:tensorflow:From /home/hr/.local/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
9/9 [==============================] - 15s 2s/step - loss: 0.3520 - acc: 0.8566 - val_loss: 0.1200 - val_acc: 0.9575
Epoch 2/20
9/9 [==============================] - 18s 2s/step - loss: 0.0974 - acc: 0.9688 - val_loss: 0.0397 - val_acc: 0.9937
Epoch 3/20
9/9 [==============================] - 16s 2s/step - loss: 0.1032 - acc: 0.9757 - val_loss: 0.0459 - val_acc: 0.9825
Epoch 4/20
9/9 [==============================] - 13s 1s/step - loss: 0.0263 - acc: 0.9931 - val_loss: 0.0535 - val_acc: 0.9900
Epoch

In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# loading haarcascade_frontalface_default.xml
face_model = cv2.CascadeClassifier(
    '../input/haar-cascades-for-face-detection/haarcascade_frontalface_default.xml')
model = load_model('masknet.h5')
mask_label = {0: 'MASK', 1: 'NO MASK'}
color_label = {0: (0, 255, 0), 1: (255, 0, 0)}

def detect():
    # Read in an image
    # img = cv2.imread(f'../input/face-mask-detection/images/maksssksksss{image_id}.png')
    ret, img = video_capture.read()
    img = cv2.cvtColor(img, cv2.IMREAD_GRAYSCALE)
    # Detect faces
    faces = face_model.detectMultiScale(img)
    new_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # colored output image
    for i in range(len(faces)):
        (x, y, w, h) = faces[i]
        crop = new_img[y:y + h, x:x + w]
        crop = cv2.resize(crop, (128, 128))
        crop = np.reshape(crop, [1, 128, 128, 3]) / 255.0
        mask_result = model.predict(crop)
        color = color_label[mask_result.argmax()]
        cv2.putText(
            new_img,
            mask_label[mask_result.argmax()],
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            color,
            2
        )
        cv2.rectangle(new_img, (x, y), (x+w, y+h), color, 2)
        cv2.imshow('Video', frame)

video_capture = cv2.VideoCapture(0)
while True:
    detect()

error: OpenCV(4.5.1) /tmp/pip-req-build-ms668fyv/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
